In [ ]:
import pandas as pd
import array, csv, datetime, glob, math, os, random

def LonLatToPixelXY(lonlat):
    (lon, lat) = lonlat
    x = (lon + 180.0) * 256.0 / 360.0
    y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    return [x, y]

def YearMonthDayToEpoch(year, month, day):
    return (datetime.datetime(int(year), int(month), int(day)) - datetime.datetime(1970, 1, 1)).total_seconds()

class GoogleSheetRow():
    columns = [
        'Enabled', 'Share link identifier', 'Category', 'Name', 'Credits', 'Base layer', 'Custom slider ticks', 'Start date',
        'End date', 'Step', 'URL', 'Scaling', 'Color Scaling', 'Map Type', 'Color', 'External GeoJSON', 'Name Key', 'Show Graph',
        'Graph Title', 'Graph Plot Colors', 'Graph Plots First Visible', 'Graph X-Axis Label', 'Graph Y-Axis Label', 'Graph Y-Axis Min',
        'Graph Y-Axis Max', 'Graph X-Axis Label Interval', 'Legend Content', 'Legend Key', 'Load Data Function', 'Set Data Function', 
        'Number of Levels', 'Number of Attributes', 'Vertex Shader', 'Fragment Shader', 'Draw Function', 'Playback Rate',
        'Master Playback Rate', 'Colormap Src', 'Layer Description', 'Featured Themes', 'Draw Options', 'Set Data Options', 'Extras Options',
        'Draw Order', 'Timeline Type', 'Layer Constraints', 'Mapbox', 'Draw Layer Function'
    ]
    key_values = {}
    def __init__(self, _dict = {}):
        for column in self.columns:
            self.key_values[column] = ''
        for key in _dict.keys():
            if key in self.columns:
                self.key_values[key] = _dict[key]
    def set_column(self, key, value):
        if key in self.columns:
            self.key_values[key] = value
    def print_row(self):
        row = []
        for column in self.columns:
            row.append(self.key_values[column])
        print("\t".join(row))
        


In [ ]:
fnames = sorted(glob.glob('../data/making-a-fires-map/DL_FIRE_SV-C2_478326/*.csv'))
fnames

In [ ]:
df = pd.read_csv(fnames[0])

In [ ]:
df.columns

In [ ]:
df['confidence'].unique()

In [ ]:
points = []
df = pd.read_csv(fnames[0])
for idx, row in df.iterrows():
    if row['confidence'] != 'l':
        x,y = LonLatToPixelXY((row['longitude'], row['latitude']))
        year,m,d = row['acq_date'].split('-')
        epoch = YearMonthDayToEpoch(year,m,d)
        points += [x,y,epoch]
df = pd.read_csv(fnames[1])
for idx, row in df.iterrows():
    if row['confidence'] != 'l':
        x,y = LonLatToPixelXY((row['longitude'], row['latitude']))
        year,m,d = row['acq_date'].split('-')
        epoch = YearMonthDayToEpoch(year,m,d)
        points += [x,y,epoch]
        

In [ ]:
len(points)/3

In [ ]:
array.array('f', points).tofile(open('../data/making-a-fires-map/making_a_fires_map.bin', 'wb'))

In [ ]:
!du -hs ../data/making-a-fires-map/making_a_fires_map.bin

In [ ]:
!scp ../data/making-a-fires-map/making_a_fires_map.bin timemachine2:/t/earthtime.org/app/data/making-earthtime-maps


In [ ]:
# Share link identifiers should be unique. Would be great if a DB just handled this... 
share_link_identifer = 'making_a_fires_map_example_1_' + str(random.getrandbits(16))
category = 'Making EarthTime Maps'
name = 'Making a fires map (Example 1)'
url = 'https://tiles.earthtime.org/making-earthtime-maps/making_a_fires_map.bin'
layer_args = {
    'Enabled': 'TRUE',
    'Share link identifier': share_link_identifer,
    'Category': category, 
    'Name': name, 
    'Credits': 'NASA', 
    'Base layer': 'bdrk', 
    'Start date': '20180101',
    'End date': '20231231', 
    'URL': url, 
    'Map Type': 'vector',
    'Load Data Function': 'WebGLVectorTile2.prototype._loadData',
    'Set Data Function': 'WebGLVectorTile2.prototype._setBufferData',
    'Number of Levels': '0',
    'Number of Attributes': '3',
    'Vertex Shader': 'WebGLVectorTile2.viirsVertexShader',
    'Fragment Shader': 'WebGLVectorTile2.viirsFragmentShader',
    'Draw Function': 'WebGLVectorTile2.prototype._drawViirs',
    'Playback Rate': '8',
    'Master Playback Rate': '16',
    'Draw Options': '{"color": [209, 56, 18]}',
    'Draw Order': '700',
}
row = GoogleSheetRow(layer_args)
row.print_row()

In [ ]:
# Customizing the time span
share_link_identifer = 'making_a_fires_map_example_2_' + str(random.getrandbits(16))
category = 'Making EarthTime Maps'
name = 'Making a fires map (Example 2)'
span = 1*24*60*60 # (Number of days) * (24 hours) * (60 minutes) * (60 seconds) 
# This should be wherever you uploaded your data to
url = 'https://tiles.earthtime.org/making-earthtime-maps/making_a_fires_map.bin'
layer_args = {
    'Enabled': 'TRUE',
    'Share link identifier': share_link_identifer,
    'Category': category, 
    'Name': name, 
    'Credits': 'NASA', 
    'Base layer': 'bdrk', 
    'Start date': '20180101',
    'End date': '20230101', 
    'URL': url, 
    'Map Type': 'vector',
    'Load Data Function': 'WebGLVectorTile2.prototype._loadData',
    'Set Data Function': 'WebGLVectorTile2.prototype._setBufferData',
    'Number of Levels': '0',
    'Number of Attributes': '3',
    'Vertex Shader': 'WebGLVectorTile2.viirsVertexShader',
    'Fragment Shader': 'WebGLVectorTile2.viirsFragmentShader',
    'Draw Function': 'WebGLVectorTile2.prototype._drawViirs',
    'Playback Rate': '8',
    'Master Playback Rate': '16',
    'Draw Options': '{"color": [187, 16, 8], "span": '+str(span)+'}',
    'Draw Order': '700',
}
row = GoogleSheetRow(layer_args)
row.print_row()

In [ ]:
points_by_year = {
    '2018': [],
    '2019': [],
    '2020': [],
    '2021': [],
    '2022': [],
    '2023': []
}
df = pd.read_csv(fnames[0])
for idx, row in df.iterrows():
    if row['confidence'] != 'l':
        x,y = LonLatToPixelXY((row['longitude'], row['latitude']))
        year,m,d = row['acq_date'].split('-')
        epoch = YearMonthDayToEpoch(year,m,d)
        points_by_year[year] += [x,y,epoch]
df = pd.read_csv(fnames[1])
for idx, row in df.iterrows():
    if row['confidence'] != 'l':
        x,y = LonLatToPixelXY((row['longitude'], row['latitude']))
        year,m,d = row['acq_date'].split('-')
        epoch = YearMonthDayToEpoch(year,m,d)
        points_by_year[year] += [x,y,epoch]


In [ ]:
for key in points_by_year.keys():
    print(key, len(points_by_year[key]))

In [ ]:
for key in points_by_year.keys():
    array.array('f', points_by_year[key]).tofile(open('../data/making-a-fires-map/making_a_fires_map_'+ key +'.bin', 'wb'))
    cmd = f'scp ../data/making-a-fires-map/making_a_fires_map_{key}.bin timemachine2:/t/earthtime.org/app/data/making-earthtime-maps'
    !$cmd


In [ ]:
# Produce 6 layers showing total for each year
category = 'Making EarthTime Maps'
span = 365*24*60*60 # (Number of days) * (24 hours) * (60 minutes) * (60 seconds) 
layer_args = {
    'Enabled': 'TRUE',
    'Share link identifier': '',
    'Category': category, 
    'Name': '', 
    'Credits': 'NASA', 
    'Base layer': 'bdrk', 
    'Start date': '',
    'End date': '', 
    'URL': '', 
    'Map Type': 'vector',
    'Load Data Function': 'WebGLVectorTile2.prototype._loadData',
    'Set Data Function': 'WebGLVectorTile2.prototype._setBufferData',
    'Number of Levels': '0',
    'Number of Attributes': '3',
    'Vertex Shader': 'WebGLVectorTile2.viirsVertexShader',
    'Fragment Shader': 'WebGLVectorTile2.viirsFragmentShader',
    'Draw Function': 'WebGLVectorTile2.prototype._drawViirs',
    'Playback Rate': '',
    'Master Playback Rate': '',
    'Draw Options': '{"color": [187, 16, 8], "span": '+str(span)+'}',
    'Draw Order': '700',
}
for key in points_by_year.keys():
    share_link_identifer = f'making_a_fires_map_example_{key}_' + str(random.getrandbits(16))
    name = f'Making a fires map ({key})'
    url = f'https://tiles.earthtime.org/making-earthtime-maps/making_a_fires_map_{key}.bin'
    layer_args['Share link identifier'] = share_link_identifer
    layer_args['Name'] = name
    layer_args['URL'] = url
    layer_args['Start date'] = f'{key}0101'
    layer_args['End date'] = f'{key}1231'
    row = GoogleSheetRow(layer_args)
    row.print_row()